In [595]:
import numpy as np
import sklearn
import pandas as pd
from pandas import Series
import os
import datetime
from sklearn.decomposition import *
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

In [596]:
os.chdir('/Users/admin/Downloads')

In [597]:
data=pd.read_csv('RawData.csv', sep=",")

In [598]:
data.columns

Index([u'Ntime', u'Time', u'Closing Price', u'Open Price', u'High price',
       u'Low Price', u'Volume', u'MACD', u'CCI', u'ATR', u'BOLL_MID', u'EMA20',
       u'MA10', u'MTM6', u'MA5', u'MTM12', u'ROC', u'SMI', u'WVAD',
       u'US Dollar Index', u'HIBOR'],
      dtype='object')

In [592]:
df = pd.DataFrame(data['Volume'])
df.index = data['Ntime']

In [602]:
data1 = data.copy()
del data1['Ntime']
data1.drop(['Time', 'Closing Price'], axis=1, inplace=True)

In [603]:
data1.head()

,Open Price,High price,Low Price,Volume,MACD,CCI,ATR,BOLL_MID,EMA20,MA10,MTM6,MA5,MTM12,ROC,SMI,WVAD,US Dollar Index,HIBOR
0,21785.3906,21938.1992,21555.5293,254858.26,-646.053200,-193.115118,546.4805,23344.24592,22956.70941,22497.96286,-751.5703,22187.92768,-887.8516,-5.754486,-0.069509,-401619.3698,71.99,1.6
1,21389.4902,21742.0703,21163.5703,272528.48,-702.282620,-186.955957,578.5000,23233.30243,22793.47797,22289.66071,-1392.3806,21909.45156,-1786.9102,-7.872373,-0.068201,-507305.2571,72.73,1.6
2,21402.1699,21534.0508,21344.8496,179634.42,-723.891770,-133.742667,291.2715,23123.37093,22663.03438,22152.28180,-1031.8496,21703.08164,-1634.1699,-8.153977,-0.058111,-398848.6140,72.71,1.6
3,21402.6992,21916.2109,21402.6992,196457.92,-693.544830,-77.445777,513.5117,23033.50750,22591.60830,22069.02789,-129.2891,21677.22382,-1412.7403,-3.880007,-0.035303,-264228.1697,72.71,1.6
4,21632.6992,21684.2109,21098.8398,201519.68,-717.087314,-108.634123,814.2207,22911.84674,22461.05613,21919.61285,-881.1993,21500.98396,-1576.7989,-6.703666,-0.055006,-518512.8442,72.96,1.6


In [604]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from keras import losses
from keras.utils.np_utils import to_categorical
import keras

In [620]:
def price_returns(dataset):
    returns = list()
    returns.append(0)
    for i in range(1, len(dataset)):
        value = (1.0*dataset[i]/dataset[i - 1]) - 1
        returns.append(value)
    return Series(returns)

In [606]:
def timeseries_to_supervised(data, lag):
    df = pd.DataFrame(data)
    df_lag = pd.DataFrame(data.iloc[:,0])
    for i in range(0,df.shape[1]):
        columns = [df.iloc[:,i].shift(j) for j in range(1, lag+1)]
        columns.append(df_lag)
        df_lag = pd.concat(columns, axis=1)
        df_lag.fillna(0, inplace=True)
    return df_lag[lag:]

In [621]:
# scale train and test data to [-1, 1]
def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)
    # transform train
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    # transform test
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled

In [622]:
# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
    new_row = [x for x in X] + [value]
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]

In [666]:
data2 = data1.copy()
data2.drop(['High price', 'Low Price', 'Volume'], axis=1, inplace=True)
data2.head()
Ret = price_returns(data2['Open Price'])
data2['Returns'] = Ret
data2.head()
data3 = data2.copy()
data3.drop(['Open Price'], axis = 1, inplace = True)
data3.head()
cols = data3.columns.tolist()
cols = cols[-1:] + cols[:-1]
data3 = data3[cols]
data3.head()
data4 = data1.copy()
Ret = price_returns(data1['Open Price'])
data4['Returns'] = Ret
data4.drop(['Open Price'], axis = 1, inplace = True)
cols = data4.columns.tolist()
cols = cols[-1:] + cols[:-1]
data4 = data4[cols]

In [676]:
def run_index_scaled(timestep, repeats, n_batch, n_epochs, n_neurons, train_up, train_low, test_up, test_low):
    '''timestep, repeats, n_batch, n_epochs, n_neurons, train_up, train_low, test_up, test_low'''
    supervised = timeseries_to_supervised(data3, timestep)
    # split data into train and test-sets
    train, test = supervised.values[train_low:train_up], supervised.values[test_low:test_up]
    # transform the scale of the data
    scaler, train_scaled, test_scaled = scale(train, test)
    # fit and evaluate model
    for i in range(repeats):
        scores, predictions, expected = fit_lstm_index_scaled(train_scaled, test_scaled, scaler, n_batch, n_epochs, n_neurons, timestep)
        plt.plot(scores['train'], color='blue', label = 'Train RMSE')
        plt.plot(scores['test'], color='orange', label = 'Test RMSE')
        print('%d) TrainScore=%f, TestScore=%f' % (i, scores['train'].iloc[-1], scores['test'].iloc[-1]))
    plt.savefig('epochs_diagnostic.png')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show() 
    return scores, predictions, expected

In [694]:
from keras import optimizers
from keras.callbacks import TensorBoard

def fit_lstm_index_scaled(scaled_train, scaled_test, scaler, batch_size, nb_epoch, neurons, timestep):
    X, y = scaled_train[:, 0:scaled_train.shape[1]-1], scaled_train[:, -1]
#   X = X.reshape(X.shape[0], scaled_train.shape[1]-1, 1)
    X = X.reshape(X.shape[0], timestep, (scaled_train.shape[1]-1)/timestep)
    # prepare model
    dropout = 0.3
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True, dropout=dropout))
    model.add(Dense(1))
    adam = optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0005)
    model.compile(loss = 'mean_squared_error', optimizer = adam)
#   model.compile(loss = 'mean_absolute_percentage_error', optimizer = adam)
    print(model.summary())
    initial_weights = model.get_weights()
    print('Initial weights:\n', initial_weights)
    # fit model
    train_rmse, test_rmse = list(), list()
    #tb_callback = keras.callbacks.TensorBoard(log_dir='/Users/admin/Documents/RL_portf_optimisation', histogram_freq=2, write_graph=True)
    #tb_callback.set_model(model)
    #callbacks = []
    #callbacks.append(tb_callback)
    for i in range(nb_epoch-1):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
        model.reset_states()
        # evaluate model on train data
        modtrain, _,_=evaluate_index_scaled(model, scaled_train, scaler, batch_size, timestep)
        train_rmse.append(modtrain)
        model.reset_states()
        # evaluate model on test data
        modtest, _,_=evaluate_index_scaled(model, scaled_test, scaler, batch_size, timestep)
        test_rmse.append(modtest)
        model.reset_states()
        
    model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
    model.reset_states()
    # evaluate model on train data
    modtrain, _,_=evaluate_index_scaled(model, scaled_train, scaler, batch_size, timestep)
    train_rmse.append(modtrain)
    model.reset_states()
    # evaluate model on test data
    modtest, predictions,expected=evaluate_index_scaled(model, scaled_test, scaler, batch_size, timestep)
    test_rmse.append(modtest)
    model.reset_states()
    scores = pd.DataFrame()
    scores['train'], scores['test'] = train_rmse, test_rmse
    return scores, predictions, expected

In [689]:
# evaluate the model on a dataset, returns RMSE in transformed units
def evaluate_index_scaled(model, scaled_data, scaler, batch_size, timestep):
    # separate
    X, y = scaled_data[:,0:scaled_data.shape[1]-1], scaled_data[:,-1]
    # reshape X.shape[0], timestep, scaled_train.shape[1]-1
    reshaped = X.reshape(X.shape[0], timestep, (scaled_data.shape[1]-1)/timestep)
    # forecast dataset
    output = model.predict(reshaped, batch_size=batch_size)
    # invert data transforms on forecast
    predictions = list()
    expected = list()
    for i in range(len(output)):
        yhat = output[i,0]
        ytrue = y[i]
#        ytrue = y[i,0]
# invert scaling
        yhat = invert_scale(scaler, X[i], yhat)
        ytrue = invert_scale(scaler, X[i], ytrue)
        predictions.append(yhat)
#       expected.append(ytrue.astype(int))
        expected.append(ytrue)
        # report performance
    plt.plot(expected, color = 'blue', label = 'True values')
    plt.plot(predictions, color = 'orange', label = 'Predicted values')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    f_score = mean_squared_error(expected, predictions)
    return f_score, predictions, expected

In [670]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [671]:
def signal(y_pred):
    signal = list()
    for i in range(0,len(y_pred)):
        if(y_pred[i]>0):
            signal.append(1)
        else:
            signal.append(-1)
    return Series(signal)

In [672]:
def transaction_cost(sig):
    count = 0
    for i in range(1, len(sig)):
        if(sig[i]!=sig[i-1]):
            count +=1
    return count*0.0002

In [673]:
run_index_scaled.__doc__

'timestep, repeats, n_batch, n_epochs, n_neurons, train_up, train_low, test_up, test_low'

In [ ]:
s = signal(pred)
plt.plot((np.ones(len(s))*exp).cumsum(), label = 'Long Strategy')
plt.plot((-np.ones(len(s))*exp).cumsum(), label = 'Short Strategy')
plt.plot((s*exp).cumsum(), label = 'Our Strategy')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()
a=s*exp
((np.mean(a)-(transaction_cost(s)/len(s)))/(np.sqrt(len(s))*np.std(a)))*np.sqrt(250)

In [ ]:
scores_n, pred, exp = run_index_scaled(2, 1, 10, 50, 4, 1500, 0, 2000, 1500)

In [ ]:
scores_n, pred, exp = run_index_scaled(2, 2, 10, 50, 4, 1500, 0, 2000, 1500)

In [ ]:
scores_n, pred, exp = run_index_scaled(2, 1, 10, 200, 4, 1500, 0, 2000, 1500)

In [ ]:
scores_n, pred, exp = run_index_scaled(2, 1, 10, 200, 4, 1500, 0, 2000, 1500)

In [ ]:
scores_n, pred, exp = run_index_scaled(3, 1, 10, 200, 4, 1500, 0, 2000, 1500)

In [ ]:
data3.shape

In [ ]:
plt.plot(data3['Returns'])
plt.show()

In [ ]:
scores_n, pred, exp = run_index_scaled(3, 1, 10, 200, 4, 1500, 250, 2000, 1500)

In [ ]:
scores_n, pred, exp = run_index_scaled(3, 1, 10, 200, 4, 1500, 250, 1700, 1500)